## Preprocessing

In [30]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
#load in dataset
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
#display data head
application_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [31]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])
#confirm removal of EIN, NAME columns
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [34]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [36]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df["APPLICATION_TYPE"].value_counts()
application_df["APPLICATION_TYPE"].head()

0    T10
1     T3
2     T5
3     T3
4     T3
Name: APPLICATION_TYPE, dtype: object

In [38]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts().loc[lambda x: x < 500].index
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(application_types_to_replace, "Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [40]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classifications_to_replace = application_df['CLASSIFICATION'].value_counts().loc[lambda x: x < 100].index
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(classifications_to_replace, "Other")



In [42]:
application_df = application_df.drop(columns=["STATUS", "SPECIAL_CONSIDERATIONS"], errors='ignore')

In [44]:
application_df = application_df.drop(columns=["STATUS", "SPECIAL_CONSIDERATIONS"], errors='ignore')

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64

In [46]:
application_df["ASK_AMT_BINNED"] = pd.cut(application_df["ASK_AMT"],
    bins=[0, 5000, 50000, 100000, 1000000, 5000000, 100000000],
    labels=["Very Low", "Low", "Med", "High", "Very High", "Extreme"])
application_df = application_df.drop("ASK_AMT", axis=1)

In [48]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [50]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop("IS_SUCCESSFUL", axis=1)
y = application_df["IS_SUCCESSFUL"]

In [58]:
# Create a StandardScaler instances, normalize data
# Fit the StandardScaler
X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size=0.3, random_state=42)
X_scaler = StandardScaler()
# Scale the data
X_train_scaled = X_scaler.fit_transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [60]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        13,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 58,113 (227.00 KB)

 Trainable params: 57,345 (224.00 KB)

 Non-trainable params: 768 (3.00 KB)

In [62]:
# Compile the model
optimizer = Adam(learning_rate=.0005)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

In [64]:
# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, callbacks=[early_stop], verbose=1)

Epoch 1/100
751/751 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.6568 - loss: 0.6472
Epoch 2/100
 12/751 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.7333 - loss: 0.5791 

C:\Users\danie\anacondapython\Lib\site-packages\keras\src\callbacks\early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


751/751 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.7199 - loss: 0.5813
Epoch 3/100
751/751 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.7252 - loss: 0.5685
Epoch 4/100
751/751 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.7235 - loss: 0.5622
Epoch 5/100
751/751 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.7225 - loss: 0.5636
Epoch 6/100
751/751 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.7269 - loss: 0.5630
Epoch 7/100
751/751 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.7291 - loss: 0.5542
Epoch 8/100
751/751 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.7283 - loss: 0.5557
Epoch 9/100
751/751 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.7297 - loss: 0.5567
Epoch 10/100
751/751 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.7303 - loss: 0.5524
Epoch 11/100
751/751 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.7312 - loss: 0.5508
Epoch 12/100
751/751 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.7317 - loss: 0.5540
Epoch 13/100
751/751 ━━━━━━━━━━━━━━━━━━━━ 1

In [70]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

322/322 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7243 - loss: 0.5789
Loss: 0.5703, Accuracy: 0.7276


In [72]:
# Export our model to HDF5 file
model.save("AlphabetSoupCharity.h5")

## 